<a href="https://colab.research.google.com/github/Amaniokl/Restaurant-Review-Sentiment-Analysis/blob/main/Tensor_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [33]:
import pandas as pd
df = pd.read_csv("Restaurant_Reviews.csv")
len(df)

1000

In [ ]:
# Import necessary libraries
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the dataset
df = pd.read_csv("Restaurant_Reviews.csv")
df.rename(columns=lambda x: x.strip(), inplace=True)  # Clean column names
print(df.columns)

# Prepare a balanced dataset
min_samples = min(len(df[df['sentiment'] == 1]), len(df[df['sentiment'] == 0]))
df_positive = df[df['sentiment'] == 1].sample(min_samples, random_state=42)
df_negative = df[df['sentiment'] == 0].sample(min_samples, random_state=42)
df_balanced = pd.concat([df_positive, df_negative])
df_balanced['label'] = df_balanced['sentiment']

# Split into training and testing sets
train_data, test_data = train_test_split(df_balanced, test_size=0.33, random_state=42)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
train_dataset = Dataset.from_pandas(train_data[['Review', 'label']])
test_dataset = Dataset.from_pandas(test_data[['Review', 'label']])

# Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the data
def tokenize_data(example):
    return tokenizer(example['Review'], truncation=True, padding='max_length', max_length=256)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

# Set the format for PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Define evaluation metrics
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
    save_total_limit=2
)

# Initialize the Train


Index(['Review', 'sentiment'], dtype='object')


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/670 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [34]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
    save_total_limit=2,
    deepspeed=None,  # Disable DeepSpeed if you're not using it
    no_cuda=False  # Set to False if you want to use GPUs, True for CPU-only
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-34-2356b0ba0b1e>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [35]:
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.131200,0.342450,0.887879,0.900000,0.872727,0.886154
2,0.064900,0.392631,0.912121,0.890805,0.939394,0.914454
3,0.029000,0.377885,0.909091,0.890173,0.933333,0.911243
4,0.002800,0.378277,0.924242,0.954545,0.890909,0.921630
5,0.003100,0.418022,0.921212,0.937107,0.903030,0.919753
6,0.003800,0.415137,0.927273,0.937888,0.915152,0.926380
7,0.000500,0.424918,0.930303,0.949367,0.909091,0.928793
8,0.012500,0.436621,0.930303,0.949367,0.909091,0.928793
9,0.000300,0.433664,0.930303,0.943750,0.915152,0.929231
10,0.008000,0.522908,0.921212,0.954248,0.884848,0.918239


Evaluation Results: {'eval_loss': 0.6123162508010864, 'eval_accuracy': 0.9181818181818182, 'eval_precision': 0.896551724137931, 'eval_recall': 0.9454545454545454, 'eval_f1': 0.9203539823008849, 'eval_runtime': 2.2186, 'eval_samples_per_second': 148.745, 'eval_steps_per_second': 2.704, 'epoch': 50.0}


In [39]:
import torch

# Check if CUDA (GPU) is available, otherwise fallback to CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Move the model to the appropriate device
model = model.to(device)

# Test the model on new examples
examples = ["Nice food","An appropriate restaurant", "I did not like this food at all"]

# Tokenize and move the inputs to the same device as the model
encoded_examples = tokenizer(examples, truncation=True, padding=True, max_length=256, return_tensors="pt")
encoded_examples = {key: value.to(device) for key, value in encoded_examples.items()}

# Make predictions
predictions = model(**encoded_examples)
predicted_classes = predictions.logits.argmax(-1).cpu().numpy()  # Move predictions back to CPU for further processing

for i in predicted_classes:
  if i == 1:
    print("Positive")
  else:
    print("Negative")
# print("Predictions:", predicted_classes)



Positive
Positive
Negative


In [37]:
!zip -r results.zip results/

updating: results/ (stored 0%)
  adding: results/checkpoint-2100/ (stored 0%)
  adding: results/checkpoint-2100/tokenizer.json (deflated 71%)
  adding: results/checkpoint-2100/scheduler.pt (deflated 56%)
  adding: results/checkpoint-2100/model.safetensors (deflated 8%)
  adding: results/checkpoint-2100/rng_state.pth (deflated 25%)
  adding: results/checkpoint-2100/special_tokens_map.json (deflated 42%)
  adding: results/checkpoint-2100/config.json (deflated 46%)
  adding: results/checkpoint-2100/vocab.txt (deflated 53%)
  adding: results/checkpoint-2100/optimizer.pt (deflated 39%)
  adding: results/checkpoint-2100/tokenizer_config.json (deflated 76%)
  adding: results/checkpoint-2100/trainer_state.json (deflated 85%)
  adding: results/checkpoint-2100/training_args.bin (deflated 51%)
  adding: results/checkpoint-2090/ (stored 0%)
  adding: results/checkpoint-2090/tokenizer.json (deflated 71%)
  adding: results/checkpoint-2090/scheduler.pt (deflated 55%)
  adding: results/checkpoint-2090